In [2]:
# Dependencies
import pandas as pd
import geopandas as gpd

In [58]:
# Set Variables
yr = 2013

In [59]:
# Load CalFire Geometry
fire_gdf = gpd.read_file(f'../outputs/calfire-geojson/ca_fire_{yr}.geojson')
fire_gdf.tail(1)

,ALARM_DATE,CONT_DATE,CAUSE_ID,CAUSE,GIS_ACRES,AGENCY,FIRE_NAME,geometry
296,2013-05-02,2013-05-11,14,Unknown/Unidentified,23676.66,CCO,SPRINGS,"MULTIPOLYGON (((-119.02166 34.15128, -119.0219..."


In [12]:
# Load NOAA-NCEI data
prcp_df = pd.read_csv(f'../outputs/noaa-ncei/CA_{yr}_prcp_data.csv')
# Create NOAA-NCEI geometry dataframe
stn_gdf = gpd.GeoDataFrame(prcp_df, geometry=gpd.points_from_xy(prcp_df.lon, prcp_df.lat), crs=4326)
stn_gdf.tail(1)

,station,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,...,2013-12-27,2013-12-28,2013-12-29,2013-12-30,2013-12-31,lat,lon,elev,name,geometry
1047,USW00094299,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,5.1,...,0.0,0.0,0.0,0.0,0.0,41.4836,-120.5614,1335.9,CA ALTURAS MUNI AP,POINT (-120.56140 41.48360)


In [ ]:
gdf = gpd.GeoDataFrame(index=fire_gdf.index)

In [65]:
# Setup base Geodataframe
gdf = fire_gdf#.drop(columns=['ALARM_DATE','CONT_DATE','CAUSE_ID','CAUSE','GIS_ACRES	AGENCY'])
# Loop through years to get precipitation data
for y in [yr-1,yr]:
    # Load NOAA-NCEI and create geometry dataframe
    df = pd.read_csv(f'../outputs/noaa-ncei/CA_{y}_prcp_data.csv')
    # Aggregate NOAA-NCEI precipitation data by month
    # Strip dataframe to precipitation data and transpose to do groupby
    prcp_df = df.drop(columns=['lat','lon','elev','name']).set_index('station').T
    # Convert date index to %Y-%m
    prcp_df.index = prcp_df.index.str.slice(0,7)
    # Perform aggregation using groupby on index and transpose back
    prcp_df = prcp_df.groupby(level=0).sum().round(1).T
    # Create geodataframe
    prcp_gdf = gpd.GeoDataFrame(prcp_df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
    # Filter NOAA-NCEI data with sjoin
    sjoin = fire_gdf.to_crs(32610).sjoin_nearest(prcp_gdf.to_crs(32610)).to_crs(4326).rename(columns={'station':f'STN{y}'})
    gdf = pd.concat( [gdf, sjoin.drop(columns=['ALARM_DATE','CONT_DATE','CAUSE_ID','CAUSE',
        'GIS_ACRES','AGENCY','FIRE_NAME','geometry','index_right'])],axis=1)
# Check geodataframe
gdf.tail(1)

,ALARM_DATE,CONT_DATE,CAUSE_ID,CAUSE,GIS_ACRES,AGENCY,FIRE_NAME,geometry,2012-01,2012-02,...,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12
296,2013-05-02,2013-05-11,14,Unknown/Unidentified,23676.66,CCO,SPRINGS,"MULTIPOLYGON (((-119.02166 34.15128, -119.0219...",32.9,2.8,...,16.0,0.8,7.3,0.0,2.5,0.0,0.0,0.3,5.1,0.5


In [ ]:
gdf.to_file(f'../outputs/calfire-geojson/CA_prcp_{yr}.geojson')